# Introduction #

# Early Stopping #

# Dropout #

# Learning Rate Schedules (???) #

# Weight Regularization (???) #
- l1/l2/l1_l2 :: Lasso/Ridge/Elastic

# The Dropout Trick #

We mentioned in the tutorial that dropout creates an ensemble network during training. It turns out that you can also use a model with dropout to create an ensemble during *inference*. This is a neat trick that can sometimes boost your model's performance a bit. Let's see how it works.

# Model Development #

# Conclusion #

How you construct your neural network determines the *hypothesis space*. You could think of regularization as weighting the hypothesis space towards a certain kind of solution. Early stopping favors solutions with the most "obvious" representations -- the features in the data the optimizer is most likely to discover first. Dropout favors solutions with robust representations.

There are many ways of regularizing a model.

You've learned about two of the most effective, but it's fun to learn about more!

# Keep Going #